In [3]:
##As of December 2023, you can run the whole code in the google colab with T4 GPU support
##it doesnt require any external package installation
##Dataset1 should be located under data directory of current folder
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from keras.models import load_model

In [4]:
charDict={}
#we encode the characters here
def asciize(strIn):
    global charDict
    charList=[]
    #print(strIn)
    for ch in str(strIn):
      if ch not in charDict:
        charDict[ch]=len(charDict)+1
      charList.append(charDict[ch])
    return charList

In [5]:
#Dataset1 is here to train and test the deep learning model
#Data in the dataset1 are the domain names without tld and country extensions.
dataDf = pd.read_csv('data/dataset1.csv')

In [6]:
dataDf["label"].value_counts()

1    168547
0    165568
Name: label, dtype: int64

In [7]:
#All domain names are encoded here
dataDf["encoded"]=dataDf["domainName"].apply(asciize)

In [8]:
dataDf["encoded"][0:10]

0              [1, 2, 3, 2, 4, 2, 5, 1, 6, 7, 8, 6, 3]
1    [9, 10, 3, 11, 12, 13, 12, 14, 7, 12, 9, 3, 11...
2                               [3, 6, 18, 2, 7, 2, 3]
3                   [19, 17, 20, 12, 5, 19, 12, 2, 16]
4    [20, 10, 4, 21, 17, 7, 7, 10, 8, 7, 18, 6, 1, ...
5                        [2, 7, 6, 4, 12, 5, 19, 2, 5]
6                             [7, 12, 9, 17, 7, 7, 15]
7                [3, 2, 16, 18, 15, 20, 2, 13, 12, 19]
8            [2, 11, 3, 15, 6, 22, 15, 23, 4, 2, 5, 2]
9    [9, 12, 14, 17, 3, 2, 5, 16, 17, 9, 6, 2, 7, 2...
Name: encoded, dtype: object

In [9]:
# Convert charDict and write JSON object to file, we will need it for the prediction in pretrained model
with open("data/encodeDict.json", "w") as outfile:
    json.dump(charDict, outfile);

In [11]:
# fix random seed for reproducibility
tf.random.set_seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = len(charDict)

X=dataDf["encoded"]
Y=dataDf["label"]

#this parameter sets the lenght of each embedding squence, we preffered 60
#if web site address is less than 60 char, rest will be filled with zero
max_review_length = 60
X = sequence.pad_sequences(X, maxlen=max_review_length, padding='post',truncating='post')

In [12]:
X_train=X[0:int(0.8*len(X))]
X_test=X[len(X_train):len(X)]

In [13]:
print(len(X_train))
print(len(X_test))

267292
66823


In [14]:
Y_train=Y[0:int(0.8*len(Y))]
Y_test=Y[len(Y_train):len(Y)]

In [15]:
print(len(Y_train))
print(len(Y_test))

267292
66823


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
# create the model
embedding_vecor_length = 8
model = Sequential()
#top_words are the parameter to set maximum number of the characters
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#each epoch takes around 1 minutest in google colam t4 gpu
history=model.fit(X_train, Y_train, epochs=125, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [18]:
model.save("modelTrained.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Opening JSON file to get character dictionary
charDictNew={}
with open('data/encodeDict.json') as json_file:
    charDictNew = json.load(json_file)

In [20]:
def asciize(strIn):
    global charMap
    charList=[]
    for ch in str(strIn):
      charList.append(charDictNew[ch])
    return charList

In [21]:
def getDomainName(url):
    dom=url
    if "." in url:
            urlList=url.split(".")
            ln=len(urlList)
            if ln==2:
                dom=urlList[0]
                return dom
            elif ln==3:
                dom=urlList[0]
                return dom
            elif ln==4:
                dom=urlList[1]
                return dom
            elif not dom.isnumeric():
                return dom

    else:
        return dom

In [25]:
def predictUrl(url):
  url=getDomainName(url)
  arr = [np.array(asciize(url))]
  my_test = sequence.pad_sequences(arr, maxlen=60, padding='post',truncating='post')
  testScore=model.predict(my_test)
  return testScore[0][0]

In [23]:
max_review_length = 60
# returns a compiled model
# identical to the previous one
model = load_model('modelTrained.h5')

In [43]:
#Then you can make predictions as;
predictUrl("www.gmail.com")

1/1 [==============================] - 0s 19ms/step


0.78515816